## 2023/10/24
## VS codeでgifを作る場合のスクリプト

問題点とその改善

- nglview.download_imageを使う場合，ブラウザのjupyter notebookなら問題ないがVS CODEだといちいちポップアップが出てしまう．
- 最初からgifを作れるnglviewの関数はそもそも動かない
- render_image()を使って保存する．この時，わざわざ別のthreadを立ち上げないといけないというのがいかに書いてある。


(まずはここを見よ)[https://github.com/nglviewer/nglview/issues/1017]
https://github.com/nglviewer/nglview/discussions/1002 

履歴
---------------

一回先生にデータを送る際に使ったものが
~/works/research/jsr_utokyo/cpmd/14_pg/ppg725/make_gif.ipynb
にあり，それをコピーしてきたもの．




In [7]:
#
# * nglviewからのデータのDL
# * 適当にxyzデータを読み込む

import ase.io
import ase 
import nglview as nv

traj = ase.io.read("../c++/traj/gromacs_pg_1ns_dt50fs_300.xyz",index=":")
print(traj)

[Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120', pbc=True, cell=[19.389799118041992, 19.389799118041992, 19.389799118041992]), Atoms(symbols='C180H480O120

In [28]:
view = nv.show_asetraj(traj, gui=True)
view

NGLWidget(max_frame=299)

In [38]:
def save_image(view=view,traj=traj):
    for frame in np.arange(0,len(traj),100):
        view.frame = int(frame)
        sleep(0.2)
        img = view.render_image(trim=True,factor=4)
        while not img.value:
            sleep(0.1)
        print(img.value)
        with open(f"img_{frame}.png", "wb") as f:
            f.write(img.value)    

In [39]:
import threading
thread = threading.Thread(
    target=save_image,
    args=(view, traj)
)
thread.daemon = True
thread.start()


In [32]:
#
# * nglviewからのデータのDL
#
'''

'''

import numpy as np
from time import sleep

def save_image(view=view,frame=0):
    view.frame = int(frame)
    sleep(0.2)
    img = view.render_image()
    while not img.value:
        sleep(0.1)
    print(img.value)
    with open("img.png", "wb") as f:
        f.write(img.value)    


# save frames every 10 step
for frame in np.arange(0,len(traj),100):
    print(frame)
    # save_image(view,frame)
    # img=view.render_image()
    # while not img.value:
    #     sleep(0.1)
    #     print("wait :: ")
    

    view.download_image(filename="0image{}.png".format(frame), trim=True) # need trim=true for truncate white space
    # make sure to let NGL spending enough time to render before going to next frame
    sleep(0.2)



0
100
200


In [13]:
from nglview.contrib.movie import MovieMaker
movie = MovieMaker(view,output="my.gif", in_memory=True)
movie

In [1]:
#
# simple code to make gif of nglview
#
# http://nglviewer.org/nglview/latest/_modules/nglview/contrib/movie.html

# make sure to change your web browser option to save files as default (vs open file by external program)
# NGLView will render each snapshot and save image to your web browser default download location

from time import sleep
import numpy as np
import os
import shutil
import moviepy.editor as mpy
from PIL import Image

# https://ambermd.org/tutorials/analysis/tutorial_notebooks/nglview_movie/index.html



In [10]:


# In my case, my default download folder is /Users/amano/var/chrome/0image{}.png
# !! caution !! change here !!
dwn_dir = "/Users/amano/works/research/tmp/figure500/" 

template_dir =dwn_dir+"tmp_dir/"
os.mkdir(template_dir)

for i in np.arange(0,500):
    shutil.move(dwn_dir+"figure_{}.png".format(str(i)), template_dir)

# In my case, my default download folder is /Users/amano/var/chrome/0image{}.png
# caution !! change here !!
template=template_dir+"figure_{}.png"

# get all (sorted) image files
imagefiles = [template.format(str(i)) for i in np.arange(0,500)]

# 実際にpicturesにappendする
pictures=[]
for i in np.arange(0,500):
    img=Image.open(template.format(str(i)))
    pictures.append(img)

#gifアニメを出力する
pictures[0].save("test_animation.gif",save_all=True, append_images=pictures[1:],\
optimize=False, duration=125, loop=0)


# 最後に中間ファイルを削除する．
# shutil.rmtree(template_dir)

#durationは１枚の画像を表示する期間をミリ秒単位で表しています。
#つまりduration=500で４枚の画像を合体させると500×4=2000ミリ秒⇒2秒の動画を作成することになります。
# frame_per_secに直すには，frame_per_sec*duration=1000 に注意．
# fps=8ならduration=125

# make a gif file

#frame_per_second = 8
#im = mpy.ImageSequenceClip(imagefiles, fps=frame_per_second)
#print(im)

# im.save(‘/Users/amano/var/chrome/mytest.gif’
#              save_all=True, append_images=images[1:], optimize=False, duration=40, loop=0)

#im.write_gif(‘my_movie.gif’, fps=frame_per_second)
#im.close()

# nglviewのを使った場合

# http://nglviewer.org/nglview/latest/_api/nglview.contrib.movie.html
# from nglview.contrib.movie import MovieMaker
# download_folder = "/Users/amano/var/chrome"
# output=output,
# download_folder=download_folder,
# mov = MovieMaker(view2,   step=100 , output="my_test.gif", timeout=0.2, in_memory=False)
#mov = MovieMaker(view2,   step=10 ,output=‘my.avi’, in_memory=True, moviepy_params=moviepy_params)
# mov.make()